In [136]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [137]:
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [322]:
from BatteryProject.ModelTwo import trainer

trainer = trainer.Trainer(deep=20,offset=100)
trainer.features = {
        'disc_capa' : 'summary_discharge_capacity.csv',
        'dis_ener' : 'summary_discharge_energy.csv',
        'temp_avg' : 'summary_temperature_average.csv',
        'char_capa' : 'summary_charge_capacity.csv'}

print(f"offset: {trainer.offset} deep: {trainer.deep} classes: {trainer.classes}")

# récupère les données
trainer.get_data(trainer.features);

offset: 100 deep: 20 classes: [550]


In [323]:
print(f"X_train.shape: {trainer.X_train.shape} y_train.shape: {trainer.y_train.shape}")

X_train.shape: (93, 20, 4) y_train.shape: (93,)


In [324]:
model = trainer.initialize_model_CNN()
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_51 (Conv1D)          (None, 20, 32)            544       
                                                                 
 max_pooling1d_39 (MaxPoolin  (None, 10, 32)           0         
 g1D)                                                            
                                                                 
 conv1d_52 (Conv1D)          (None, 10, 32)            3104      
                                                                 
 max_pooling1d_40 (MaxPoolin  (None, 5, 32)            0         
 g1D)                                                            
                                                                 
 conv1d_53 (Conv1D)          (None, 5, 32)             2080      
                                                                 
 max_pooling1d_41 (MaxPoolin  (None, 2, 32)          

In [325]:
trainer.fit(model)

Epoch 1/100
5/5 [==============================] - 1s 56ms/step - loss: 671486.7500 - mse: 671486.7500 - val_loss: 879534.0625 - val_mse: 879534.0625
Epoch 2/100
5/5 [==============================] - 0s 14ms/step - loss: 648002.5000 - mse: 648002.5000 - val_loss: 844820.5000 - val_mse: 844820.5000
Epoch 3/100
5/5 [==============================] - 0s 14ms/step - loss: 610487.5625 - mse: 610487.5625 - val_loss: 791290.6250 - val_mse: 791290.6250
Epoch 4/100
5/5 [==============================] - 0s 16ms/step - loss: 553734.9375 - mse: 553734.9375 - val_loss: 711069.0000 - val_mse: 711069.0000
Epoch 5/100
5/5 [==============================] - 0s 15ms/step - loss: 470298.9375 - mse: 470298.9375 - val_loss: 596600.9375 - val_mse: 596600.9375
Epoch 6/100
5/5 [==============================] - 0s 15ms/step - loss: 358032.5000 - mse: 358032.5000 - val_loss: 455178.8125 - val_mse: 455178.8125
Epoch 7/100
5/5 [==============================] - 0s 14ms/step - loss: 230875.5000 - mse: 230875.50

In [327]:
error = trainer.eval(model)
print(f"MSE: {round(error[0],2)}  RMSE: {round(np.sqrt(error[0]),2)}")
error

MSE: 166547.36  RMSE: 408.1


[166547.359375, 166547.359375]